# Testing Run Length Encoding


## Imports


In [64]:
from importlib.util import spec_from_loader, module_from_spec
from importlib.machinery import SourceFileLoader
from scipy.io import wavfile
from glob import glob
import numpy as np
import sys, os

# Import Encode
spec = spec_from_loader("encode", SourceFileLoader("encode", "../../encode"))
encode = module_from_spec(spec)
spec.loader.exec_module(encode)

# Import Decode
spec = spec_from_loader("decode", SourceFileLoader("decode", "../../decode"))
decode = module_from_spec(spec)
spec.loader.exec_module(decode)

## Function Definitions


In [65]:
def encode_via_rle(rle_l_raw: list):
    """This algorithm will search for contiguous values within the
       array. When the rle_location_count is greater than the value
       65530, the count is reduced by this value in order to
       prevent an overflow of an unsigned 16-bit integer. This allows
       for the data to be stored with 2 bytes when the format of the
       array is a known value in advance of decoding this format. The
       choice of integer 65530 is an arbitrary value less than that of
       the maximum value of an unsigned 16-bit integer (65536).

    Args:
        rle_l_raw (list): This is a list of integer values to be
                          encoded.

    Returns:
        index_array (list): This is the list of run length encoded
                            values.
        rle_locations (list): This is a list of locations of elements
                              that are repeated that are present in the
                              array of indices.
    """
    initial_index = 0
    second_index = 1
    frequency = 0
    index_array = []
    rle_locations = []
    rle_location_count = 0

    while second_index < len(rle_l_raw):
        if rle_l_raw[initial_index] == rle_l_raw[second_index]:
            index_array.append(rle_l_raw[initial_index])
            rle_locations.append(rle_location_count)

            # continue searching the breadth of the array; increasing
            # the detected frequency of the value.
            while (
                second_index <= len(rle_l_raw)
                and rle_l_raw[initial_index] == rle_l_raw[second_index]
            ):
                frequency += 1
                second_index += 1
            index_array.append(frequency)
            if rle_location_count > 65530:
                rle_location_count -= 65530
            rle_location_count += 2
        else:
            index_array.append(rle_l_raw[initial_index])
            if rle_location_count > 65530:
                rle_location_count -= 65530
            rle_location_count += 1
        frequency = 0
        initial_index = second_index
        second_index += 1
    if rle_l_raw[-1] != index_array[-2]:
        index_array.append(rle_l_raw[-1])
    return index_array, rle_locations

In [66]:
def identify_index_split(rle_locations, verbose=False):
    """This will identify where the rle_location values reset to a value
       that is below 65530.
    Args:
        rle_locations (list): This is a list of locations of values that
                              are encoded in the index_array.

    Returns:
        split_indices (list): This is a list of indices where the values
                              are reset in the rle_locations_array.
    """
    split_indices = []
    for index, value in enumerate(rle_locations):
        if index == 0:
            prev_value = value
            continue
        if value > prev_value:
            prev_value = value
        else:
            if verbose:
                print(f"index: {index}")
                print(f"prev_value: {prev_value}")
                print(f"value: {value}")
            split_indices.append(index)
            prev_value = value
    return split_indices

In [67]:
# Expand RLE Algorithm
def decode_rle(rle_locations: list, index_array: list):
    """This will expand the index array where the values have been
       compressed by run-length-encoding.

    Args:
        rle_locations (list): This is a list of locations in the
                              index_array where the locations have been
                              run-length-encoded. The subsequent values
                              of these locations are frequencies of the
                              run-length-encoded values.
        index_array (list): This is a list of values which contain
                            either signular values or run-length-encoded
                            values followed by the frequency of the
                            run-length-encoded value.
    """
    reconstructed_array = []
    rle_index = 0
    continue_current_index_past_frequency_value = False

    for current_index in range(0, len(index_array)):
        if continue_current_index_past_frequency_value:
            continue_current_index_past_frequency_value = False
            continue
        if rle_index <= (len(rle_locations) - 1):
            if current_index > 131060:  # twice the cutoff value of 65530
                rle_location_index = rle_locations[rle_index] + 131060
            elif current_index > 65530 and current_index <= 131060:
                rle_location_index = rle_locations[rle_index] + 65530
            else:  # current_index is below 65530
                rle_location_index = rle_locations[rle_index]
        if current_index != rle_location_index:
            reconstructed_array.append(index_array[current_index])
        else:
            if rle_index <= (len(rle_locations) - 1):
                run_length_expanded_l = [
                    index_array[rle_location_index]
                    for x in range((index_array[rle_location_index + 1]) + 1)
                ]
                reconstructed_array.extend(run_length_expanded_l)
                rle_index += 1
                continue_current_index_past_frequency_value = True
    return reconstructed_array

In [68]:
def format_encoded_rle_to_bytes(index_array: list, rle_locations: list):
    """This is the encoding driver of the run_length_encoded array.

    Args:
        index_array (list): This is the compressed list of values which
                            include run-length-encoded values followed
                            by their frequency.
        rle_locations (list): This is a list of index locations that are
                              run-length-encoded in the index array.
                              The frequency of the run-length-encoded
                              value is defined as the
                              index_array[rle_locations[current_rle_index] + 1]
                              or the subsequent value from the
                              run-length-encoded value in the
                              index_array.

    Returns:
        format_encoded_rle_bytes (bytes): This is the string of bytes of
                                          the formatted
                                          run-length-encoded array.
    """
    format_encoded_rle = [len(rle_locations)]
    format_encoded_rle.extend(rle_locations)
    format_encoded_rle.extend(index_array)
    format_encoded_rle = np.array(format_encoded_rle, dtype=np.uint16)
    format_encoded_rle_bytes = format_encoded_rle.tobytes()
    return format_encoded_rle_bytes

In [69]:
def parse_formatted_rle_bytes(format_encoded_rle_bytes: bytes):
    """This is the encoding driver of the run_length_encoded array.

    Args:
        format_encoded_rle_bytes (bytes): This is the compressed list of
                                          values in the form of a byte
                                          array. The order of the values
                                          is as following: length of the
                                          run-length-encoded locations
                                          array, the run-length-encoded
                                          locations array, and the
                                          index_array.

    Returns:
        rle_locations (list): This is a list of unsigned 16-bit
                              integers.
        index_array (list): This is a list of unsigned 16-bit integers.
                            These values contain run-length-encoded
                            sequences where the run-length-encoded-value
                            is followed by the frequency of the value.
    """
    encoded_rle = np.frombuffer(format_encoded_rle_bytes, dtype=np.uint16)
    len_rle_locations = encoded_rle[0]

    # +1 to skip over the rle_locations length in the line below:
    rle_locations = encoded_rle[1 : len_rle_locations + 1]

    index_array = encoded_rle[(len_rle_locations + 1) :]
    return rle_locations, index_array

## Import Data & Huffman Encode Values


In [70]:
data_dir = "../../data/"
data_file_l = glob(data_dir + "*.wav")
current_file = data_file_l[0]

rate, data = wavfile.read(current_file)

data_bytes = data.tobytes()

compressed_file_path = "../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire"

encode.huffman_encoding(
    input_data=data_bytes,
    compressed_file_path="../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire",
)

with open(compressed_file_path, "rb") as fp:
    data_huffman_encoded_bytes = fp.read()
    fp.close()

# type(data_huffman_encoded_bytes)

## Testing Compressing a substring of the larger dataset


In [71]:
data_huffman_encoded_bytes_sub_string = data_huffman_encoded_bytes[:12]

rle_l_raw = []
for index, data in enumerate(data_huffman_encoded_bytes_sub_string):
    rle_l_raw.append(data)
rle_l_raw

[128, 4, 149, 135, 5, 0, 0, 0, 0, 0, 0, 125]

In [72]:
len(rle_l_raw)

12

In [73]:
sys.getsizeof(rle_l_raw)

184

In [74]:
sys.getsizeof(rle_l_raw[0])

28

In [75]:
type(rle_l_raw[0])

int

In [76]:
index_array, rle_locations = encode_via_rle(rle_l_raw)
reconstructed_array = decode_rle(rle_locations, index_array)

In [77]:
sys.getsizeof(reconstructed_array)

152

In [78]:
sys.getsizeof(reconstructed_array[0])

28

In [79]:
type(reconstructed_array[0])

int

In [80]:
reconstructed_array[0]

128

In [81]:
np.array(rle_l_raw, dtype=np.int16)

array([128,   4, 149, 135,   5,   0,   0,   0,   0,   0,   0, 125],
      dtype=int16)

In [82]:
# Test for equality in reconstructed array
not_equal = False
for index in range(0, len(rle_l_raw)):
    if rle_l_raw[index] != reconstructed_array[index]:
        not_equal = True
        print(f"Reconstructed Array is unequal at position: {index}")

if not_equal:
    print(f"Arrays are not equivalent.")
else:
    print(f"Arrays are equivalent.")

Arrays are equivalent.


In [83]:
formatted_encoded_rle_bytes = format_encoded_rle_to_bytes(index_array, rle_locations)

In [84]:
encoded_rle = np.frombuffer(formatted_encoded_rle_bytes, dtype=np.uint16)

In [85]:
rle_locations, index_array = parse_formatted_rle_bytes(formatted_encoded_rle_bytes)

In [86]:
reconstructed_array = decode_rle(rle_locations, index_array)

In [87]:
# Test for equality in reconstructed array
for index in range(0, len(rle_l_raw)):
    if rle_l_raw[index] != reconstructed_array[index]:
        print(f"Reconstructed Array is unequal at position: {index}")

In [88]:
len(formatted_encoded_rle_bytes)

20

In [89]:
len(data_huffman_encoded_bytes[:12])

12

In [90]:
formatted_encoded_rle_bytes

b'\x01\x00\x05\x00\x80\x00\x04\x00\x95\x00\x87\x00\x05\x00\x00\x00\x05\x00}\x00'

In [91]:
import pandas as pd

In [92]:
data_array = []
for index, data in enumerate(formatted_encoded_rle_bytes):
    data_array.append(data)
    print(f"index: {index}\ndata: {data}\n")
formatted_encoded_rle_bytes_pd = pd.DataFrame(data_array)

index: 0
data: 1

index: 1
data: 0

index: 2
data: 5

index: 3
data: 0

index: 4
data: 128

index: 5
data: 0

index: 6
data: 4

index: 7
data: 0

index: 8
data: 149

index: 9
data: 0

index: 10
data: 135

index: 11
data: 0

index: 12
data: 5

index: 13
data: 0

index: 14
data: 0

index: 15
data: 0

index: 16
data: 5

index: 17
data: 0

index: 18
data: 125

index: 19
data: 0



In [93]:
formatted_encoded_rle_bytes_pd.columns = ["value"]

In [94]:
formatted_encoded_rle_bytes_pd.index.name = "row"

In [95]:
formatted_encoded_rle_bytes_pd

,value
row,
0,1
1,0
2,5
3,0
4,128
5,0
6,4
7,0
8,149


In [96]:
rle_l_raw

[128, 4, 149, 135, 5, 0, 0, 0, 0, 0, 0, 125]

In [97]:
data_huffman_encoded_bytes[:12]

b'\x80\x04\x95\x87\x05\x00\x00\x00\x00\x00\x00}'

In [98]:
data_array = []
for data in data_huffman_encoded_bytes:
    data_array.append(data)

In [99]:
sys.getsizeof(data_array[0])

28

In [100]:
max(data_array)

255

In [101]:
data_array = np.array(data_array, dtype=np.int16)

In [102]:
data_array

array([128,   4, 149, ..., 101,  46,  28], dtype=int16)

In [103]:
data_array[0]

128

In [104]:
print(f"Ratio of the formatted_encoded_rle_bytes ", end="")
print(f"to data_huffman_encoded_bytes: ", end="")
print(f"{len(formatted_encoded_rle_bytes) / len(data_huffman_encoded_bytes[:12])}")

Ratio of the formatted_encoded_rle_bytes to data_huffman_encoded_bytes: 1.6666666666666667


#### Principal Components Example of the Run-Length-Encoded Problem


In [105]:
# Original file size: 197 KB
# Huffman Encoded Compressed File Size: 130 KB
# RLE Encoded Version of the Huffman Encoded Compressed File is of Size: 272 KB

In [106]:
data_dir = "../../data/"
data_file_l = glob(data_dir + "*.wav")
current_file = data_file_l[0]

In [107]:
rate, data = wavfile.read(current_file)

data_bytes = data.tobytes()

compressed_file_path = "../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire"

encode.huffman_encoding(
    input_data=data_bytes,
    compressed_file_path="../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire",
)

with open(compressed_file_path, "rb") as fp:
    data_huffman_encoded_bytes = fp.read()
    fp.close()

In [108]:
sys.getsizeof(data)

197510

In [109]:
sys.getsizeof(data_bytes)

197431

## Attempting RLE Exclusively


In [110]:
data_dir = "../../data/"
data_file_l = glob(data_dir + "*.wav")
current_file = data_file_l[0]

rate, data = wavfile.read(current_file)

data_bytes = data.tobytes()

compressed_file_path = "../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav.brainwire"

In [111]:
os.getcwd() + "/" + current_file

'/Volumes/T7/Github/Neuralink-Compression-Challenge/analysis/tutorials/../../data/102b47d9-371e-412a-8995-0dc6115ab2bb.wav'

In [112]:
data_bytes

b'\xa0\xfe`\xfe\xe0\xfe\xe0\xfe`\xff`\x02_\x00`\xff\xdf\x00\xdf\x00\xdf\x01\xa0\x04\xe0\x04`\x03\xe0\x02 \x05\xe0\x04`\x05\xe0\x04\xe0\x02\xa0\x02`\x05`\x04`\x05\xe1\x06\xe1\x06\xa1\x07\xa1\x06a\x07!\x07 \x05 \x05\xe0\x05`\x05\xe0\x05\xa1\x06\xa1\x08!\t!\t\xe1\t\xa2\x0c\xa2\n\xa1\t\xe1\x07a\x08\xa1\x08\xe1\x06!\x06\xa1\t!\n\xa1\x07!\x08!\x06\xa0\x05\xe0\x05`\x05`\x05`\x04\xe0\x04\xa1\x06\xe1\x07\xe1\x06!\x06a\x06!\x07 \x05\xdf\x01\xdf\x00\x9f\x01\xdf\x01\xa0\x02\xa0\x04!\x06\xe1\x07\xe0\x05\xa1\x06a\x06\xe0\x04\xe0\x05\xa0\x04\xe0\x04\xe0\x04\xa1\x06\xe1\x08\xa1\x06!\x06\xa1\x07\xa0\x04\xe0\x03\xe0\x02\xa0\x03a\x06\xe1\x06!\x06\xe0\x04\xe0\x05\xe1\x06\xa1\x06!\x06`\x04\xe0\x03 \x04\xa0\x02\xa0\x02_\x00`\xff\x1f\x00`\xff`\x03 \x05\xa0\x03\xe0\x02\xa0\x02\x9f\x00\xe0\xfe\x1f\x00_\x00\xdf\x00\x9f\x01`\x02\xdf\x00\xa0\xfe\x9f\x00 \xff\xe0\xfe\x9f\x00\x9f\x00_\x01`\x02\x9f\x01\x9f\x01\x1f\x01\xe0\x02`\x03 \x05\xe0\x04\xe0\x05\xa0\x04\xe0\x05\xa1\x06\xe0\x04\xa0\x04\xe0\x02\xa0\x03`\x02 \x02

In [113]:
index_array, rle_locations = encode_via_rle(data_bytes)

In [114]:
index_array

[160,
 254,
 96,
 254,
 224,
 254,
 224,
 254,
 96,
 255,
 96,
 2,
 95,
 0,
 96,
 255,
 223,
 0,
 223,
 0,
 223,
 1,
 160,
 4,
 224,
 4,
 96,
 3,
 224,
 2,
 32,
 5,
 224,
 4,
 96,
 5,
 224,
 4,
 224,
 2,
 160,
 2,
 96,
 5,
 96,
 4,
 96,
 5,
 225,
 6,
 225,
 6,
 161,
 7,
 161,
 6,
 97,
 7,
 33,
 7,
 32,
 5,
 32,
 5,
 224,
 5,
 96,
 5,
 224,
 5,
 161,
 6,
 161,
 8,
 33,
 9,
 33,
 9,
 225,
 9,
 162,
 12,
 162,
 10,
 161,
 9,
 225,
 7,
 97,
 8,
 161,
 8,
 225,
 6,
 33,
 6,
 161,
 9,
 33,
 10,
 161,
 7,
 33,
 8,
 33,
 6,
 160,
 5,
 224,
 5,
 96,
 5,
 96,
 5,
 96,
 4,
 224,
 4,
 161,
 6,
 225,
 7,
 225,
 6,
 33,
 6,
 97,
 6,
 33,
 7,
 32,
 5,
 223,
 1,
 223,
 0,
 159,
 1,
 223,
 1,
 160,
 2,
 160,
 4,
 33,
 6,
 225,
 7,
 224,
 5,
 161,
 6,
 97,
 6,
 224,
 4,
 224,
 5,
 160,
 4,
 224,
 4,
 224,
 4,
 161,
 6,
 225,
 8,
 161,
 6,
 33,
 6,
 161,
 7,
 160,
 4,
 224,
 3,
 224,
 2,
 160,
 3,
 97,
 6,
 225,
 6,
 33,
 6,
 224,
 4,
 224,
 5,
 225,
 6,
 161,
 6,
 33,
 6,
 96,
 4,
 224,
 3,
 32,
 4,
 16

In [115]:
len(data_bytes)

197398

In [116]:
sys.getsizeof(data)

197510

In [120]:
formatted_encoded_rle_bytes = format_encoded_rle_to_bytes(index_array, rle_locations)

In [118]:
sys.getsizeof(data_bytes)

197431

In [119]:
formatted_encoded_rle_bytes

b'\x00\x00\xa0\x00\xfe\x00`\x00\xfe\x00\xe0\x00\xfe\x00\xe0\x00\xfe\x00`\x00\xff\x00`\x00\x02\x00_\x00\x00\x00`\x00\xff\x00\xdf\x00\x00\x00\xdf\x00\x00\x00\xdf\x00\x01\x00\xa0\x00\x04\x00\xe0\x00\x04\x00`\x00\x03\x00\xe0\x00\x02\x00 \x00\x05\x00\xe0\x00\x04\x00`\x00\x05\x00\xe0\x00\x04\x00\xe0\x00\x02\x00\xa0\x00\x02\x00`\x00\x05\x00`\x00\x04\x00`\x00\x05\x00\xe1\x00\x06\x00\xe1\x00\x06\x00\xa1\x00\x07\x00\xa1\x00\x06\x00a\x00\x07\x00!\x00\x07\x00 \x00\x05\x00 \x00\x05\x00\xe0\x00\x05\x00`\x00\x05\x00\xe0\x00\x05\x00\xa1\x00\x06\x00\xa1\x00\x08\x00!\x00\t\x00!\x00\t\x00\xe1\x00\t\x00\xa2\x00\x0c\x00\xa2\x00\n\x00\xa1\x00\t\x00\xe1\x00\x07\x00a\x00\x08\x00\xa1\x00\x08\x00\xe1\x00\x06\x00!\x00\x06\x00\xa1\x00\t\x00!\x00\n\x00\xa1\x00\x07\x00!\x00\x08\x00!\x00\x06\x00\xa0\x00\x05\x00\xe0\x00\x05\x00`\x00\x05\x00`\x00\x05\x00`\x00\x04\x00\xe0\x00\x04\x00\xa1\x00\x06\x00\xe1\x00\x07\x00\xe1\x00\x06\x00!\x00\x06\x00a\x00\x06\x00!\x00\x07\x00 \x00\x05\x00\xdf\x00\x01\x00\xdf\x00\x00\x00\x9f\x